# MAT281 - Tarea 2

**Indicaciones**:

* En los **Ejercicio 1-8** puedes utilizar tanto `matplotlib` como `altair` según te parezca más conveniente o cómodo, en ambos casos cada gráfico debe tener elementos mínimos como:
    - Título
    - Nombre de los ejes, leyendas, etc. en formato _amigable_/_humano_, por ejemplo, si la columna del dataframe en cuestión tiene por nombre `casos_confirmados` se espera que el eje del gráfico tenga por nombre `Casos confirmados`.
    - Colores adecuados al tipo de datos.
    - Un tamaño adecuado para ver con facilidad en una pantalla con resolución HD o FullHD.
    - Cada vez que no se cumplan alguna de estos requerimientos se descontará __1 punto__ de la nota final.

* Para el **Ejercicio 9** es obligación utilizar `altair`.
* Cada ejercicio debe estar acompañado con una celda con comentarios o análisis que puedas desprender de los gráficos.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import ipywidgets as widgets

from datetime import date
from ipywidgets import interactive, interact

pd.set_option('display.max_columns', 999)
alt.data_transformers.disable_max_rows()
#alt.data_transformers.enable('data_server')
alt.themes.enable('opaque')

%matplotlib inline

**COVID-19 en Chile** 

En esta tarea exploraremos los datos de Covid-19 en Chile a profundidad. Las siguientes celdas cargarán los datos a utilizar en tu sesión. Es importante que leas la documentación de cada conjunto de datos para comprender las columnas.

In [ ]:
start_date = pd.to_datetime("2020-04-13")

In [ ]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto6
confirmados = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto6/bulk/data.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
    .dropna()
    .astype({"casos_confirmados": np.float, "tasa": np.float})
)

confirmados.head()

In [ ]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto19
activos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .loc[lambda x: x["codigo_comuna"].notnull()]
    .melt(id_vars=["region", "codigo_region", "comuna", "codigo_comuna", "poblacion"], var_name="fecha", value_name="casos_activos")
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
)

activos.head()

In [ ]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto14
fallecidos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["region"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos = fallecidos.drop(fallecidos.query("region == 'Total'").index)

fallecidos.head()

In [ ]:
fallecidos["region"].unique()

In [ ]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto10
fallecidos_etareo = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto10/FallecidosEtario.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["grupo_de_edad"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
        grupo_de_edad=lambda x: x["grupo_de_edad"].str.replace("<=39", "0-39")
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos_etareo.head()

## Ejercicio 1

(10 puntos)

Mostrar cantidad de fallecidos a la fecha por cada grupo etáreo.

In [ ]:
# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['fecha'], empty='none')

# Grafico de Lineas
line = alt.Chart(fallecidos_etareo).mark_line(interpolate='basis').encode(
    x=alt.X('fecha', title="Fecha"), 
    y=alt.Y('fallecidos:Q', title="Fallecidos"),
    color=alt.Color('grupo_de_edad:N', title="Grupo Etáreo",scale=alt.Scale(scheme='category10'))
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(fallecidos_etareo).mark_point().encode(
    x='fecha',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'fallecidos:Q', alt.value(' '))
)

text2 = alt.Chart(fallecidos_etareo).mark_text(align='left', dx=-200, dy=-100, size=20).encode(
    text=alt.condition(nearest, 'fecha', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(fallecidos_etareo).mark_rule(color='firebrick').encode(
    x='fecha',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
ev_grupo=alt.layer(
    line, selectors, points, rules, text, text2
).properties(
    title="Evolución de Fallecidos por Grupo Etáreo",
     height=300
).interactive()

ev_grupo

**Comentarios:** Podemos ver que, los grupos de mayor edad tienen mas fallecidos, excepto el grupo de personas con edades mayores o iguales a 80, esto se puede deber a que el porcentaje de la poblacion de Chile que se encuentra en este grupo etareo es muy baja. 

## Ejercicio 2

(10 puntos)

¿Qué tan variable es la población de las comunas de Chile? Considera utilizar un gráfico que resuma de buena forma la información sin agregar la variable de región o provincia.

In [ ]:
bar = alt.Chart(activos).mark_bar().encode(
    x=alt.X("comuna:N", title="Comuna"),
    y=alt.Y("poblacion:Q", title="Población"),
    tooltip=[alt.Tooltip("poblacion", title="Población"), alt.Tooltip("comuna",title="Comuna")]
)

rule = alt.Chart(activos).mark_rule(color='firebrick').encode(
    y='mean(poblacion):Q',
    size=alt.SizeValue(3)
)

(bar + rule).properties(
    title="Población por Comuna").interactive() #p!rule34 donkey kong 

In [ ]:
std=round(np.std(activos["poblacion"]),2)
mean=round(np.mean(activos["poblacion"]),2)
print(f"El promedio de población en las comunas de Chile es: {mean}") 
print(f"La desviación estandar de población en las comunas de Chile es: {std}")

**Comentarios:** El grafico y la desviación estandar nos dicen que la poblacion por comuna en Chile es muy variable. La mayoria de comunas estan por debajo del promedio de población, pero las comunas con mas habitantes estan muy por encima, lo que resulta en la alta desviacion estandar.

## Ejercicio 3

(10 puntos)

Mostrar evolución y comparación de los fallecimientos entre distintos grupos etáreos, pero que al mismo tiempo sea fácil identificar la cantidad de fallecidos total en cada fecha.

In [ ]:
cercano = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['fecha'], empty='none')

# Grafico de Lineas
lineas = alt.Chart(fallecidos).mark_line(interpolate='basis', color= "black").encode(
    x=alt.X('fecha', title="Fecha"), 
    y=alt.Y('sum(fallecidos):Q', title="Fallecidos")
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectores = alt.Chart(fallecidos).mark_point().encode(
    x='fecha',
    opacity=alt.value(0),
).add_selection(
    cercano
)

# Draw points on the line, and highlight based on selection
puntos = lineas.mark_point().encode(
    opacity=alt.condition(cercano, alt.value(1), alt.value(0))                          #GRAFICO EVOLUCION DE FALLECIDOS TOTAL
)

# Draw text labels near the points, and highlight based on selection
texto = lineas.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(cercano, "sum(fallecidos):Q", alt.value(' '))
)

texto2 = alt.Chart(fallecidos_etareo).mark_text(align='left', dx=-200, dy=-100, size=20).encode(
    text=alt.condition(cercano, "fecha", alt.value(' '))
)

# Draw a rule at the location of the selection
reglas = alt.Chart(fallecidos).mark_rule(color='firebrick').encode(
    x='fecha',
).transform_filter(
    cercano
)

# Put the five layers into a chart and bind the data
ev_total=alt.layer(
    lineas, selectores, puntos, reglas, texto, texto2
).properties(
    title = "Evolución del Total de Fallecidos",
     height=300
)

In [ ]:
ev_grupo | ev_total

**Comentarios:** Con estos graficos podemos verificar que, por fecha, el total de fallecidos es la suma de los fallecidos por grupo etáreo.

## Ejercicio 4

(10 puntos)

Mostrar en tres gráficos la evolución de casos confirmados, evolución de fallecimientos y evolución de casos activos.

In [ ]:
Casos_confirmados=alt.Chart(confirmados).mark_line().encode(
                x=alt.X("fecha", title="Fecha"),
                y=alt.Y("sum(casos_confirmados):Q", title="Casos Confirmados")
).properties(
                title="Evolución Casos Confirmados",
                
)




Ev_fallecidos=alt.Chart(fallecidos).mark_line().encode(
                x=alt.X("fecha", title="Fecha"),
                y=alt.Y("sum(fallecidos)", title="Fallecidos")
).properties(
                title="Evolucion Fallecidos",
                
)




Casos_activos=alt.Chart(activos).mark_line().encode(
                x=alt.X("fecha", title="Fecha"),
                y=alt.Y("sum(casos_activos)", title="Casos Activos"),
).properties(
                title="Evolucion Casos Activos",
                
)



Casos_confirmados | Ev_fallecidos | Casos_activos

**Comentarios:** Los casos activos y los casos confirmados, al ser valores acumulados en el tiempo, tiene sentido que sus graficas sean curvas crecientes. La evolución de los casos activos nos muestra lo rapido que aumentaron los casos en Chile, y, la reacción frente a el aumento de casos. 

## Ejercicio 5

(10 puntos)

Comparar la tasa de incidencia entre las regiones a lo largo del tiempo.

In [ ]:
highlight = alt.selection(type='single', on='mouseover',
                          fields=['region'], nearest=True)

base = alt.Chart(confirmados).encode(
    x=alt.X("fecha",title="Fecha"),
    y=alt.Y("sum(tasa):Q",title="Tasa de Incidencia"),
    color=alt.Color("region:N",title="Region",scale=alt.Scale(scheme='category20')),
    tooltip=[alt.Tooltip("sum(tasa):Q", title="Tasa de Incidencia"), alt.Tooltip("fecha",title="Fecha"), alt.Tooltip("region", title="Region")]
).properties(
    title="Evolución de Tasa de Incidencia por Region",
    width=800,
    height=600
)

points = base.mark_circle().encode(
    opacity=alt.value(0)
).add_selection(
    highlight
).properties(
    width=600
)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(3), alt.value(6))
)

points + lines

**Comentarios:** Los casos confirmados, al ser la suma acumulada a lo largo del tiempo de los casos confirmados, tiene sentido que la tasa de incidencia de cada region sean curvas crecientes. Por otro lado, es claro ver que las regiones con mas población tienden a tener una mayor tasa de incidencia, pero, hay algunas regiones que tienen una mayor tasa a pesar de tener menos habitantes, como es el caso de Magallanes o O'Higgins, ambas regiones tienen una mayor tasa de incidencia que otras regiones que tienen mas habitantes.

## Ejercicio 6

(10 puntos)

¿Hay alguna conclusión que puedas obtener rápidamente al graficar un _scatter plot_ con los casos confirmados y tasa de incidencia de cada comuna para los días 13 de abril y 6 de noviembre del 2020? Además, colorea cada punto según la región a la que pertenece y considera si es útil en el gráfico que el tamaño sea proporcional a la población.

In [ ]:
fecha1=confirmados.loc[lambda x: x["fecha"] == "2020-04-13"] 
fecha2=confirmados.loc[lambda x: x["fecha"] == "2020-11-06"]

In [ ]:
df_fecha1=alt.Chart(fecha1).mark_circle().encode(
        x=alt.X("casos_confirmados", title="Casos Confirmados"),
        y=alt.Y("tasa", title="Tasa de Incidencia"),
        color=alt.Color("region",title="Región",scale=alt.Scale(scheme='category20')),
        size="poblacion"
).properties(
        title="13 de Abril"
)

df_fecha2=alt.Chart(fecha2).mark_circle().encode(
        x=alt.X("casos_confirmados", title="Casos Confirmados"),
        y=alt.Y("tasa", title="Tasa de Incidencia"),
        color=alt.Color("region",title="Región",scale=alt.Scale(scheme='category20')),
        size="poblacion"
).properties(
        title="6 de Noviembre"
)

df_fecha1 | df_fecha2

**Comentarios:** Hay comunas con una poblacion menor o igual a 100.000 habitantes que tienen una tasa de incidencia muy grande, esto significa que hubo un mayor porcentaje de la poblacion infectada de COVID-19 en estas comunas, podemos comparar estos casos a las comunas con mas habitantes, vemos que en estas comunas mas pobladas hubieron mas casos confirmados, pero menor tasa de incedencia, porque hubo un menor porcentaje de la pobalción infectada en la comunas con mas habitantes.

## Ejercicio 7

(10 puntos)

1. Grafica la evolución de los casos activos de cada comuna en un solo gráfico. 
2. Grafica la evolución de los casos activos de cada comuna en gráficos separados por región.

Entrega los pros y contras de cada uno de estos enfoques.

In [ ]:
ev1=alt.Chart(activos).mark_line().encode(
        x=alt.X("fecha", title="Fecha"),
        y=alt.Y("casos_activos", title="Casos Activos"),
        color=alt.Color("comuna",title="Comuna",scale=alt.Scale(scheme='category20'))
).properties(
        title="Evolución de Casos Activos por Comuna"
)

ev1

In [ ]:
for i in activos.loc[:,"region"].unique():
    df=activos.loc[lambda x: x["region"]==i]
    ev_comuna=alt.Chart(df).mark_line().encode(
        x=alt.X("fecha", title="Fecha"),
        y=alt.Y("casos_activos", title="Casos Activos"),
        color=alt.Color("comuna",title="Comuna",scale=alt.Scale(scheme='category20'))
    ).properties(
        title=f"Evolución de Casos Activos por Comuna en {i}"
    )
    display(ev_comuna)

**Comentarios:** Si graficamos todas las comunas en un solo grafico, en este caso, podemos comparar la evolución de los casos activos de las comunas con mas casos activos a nivel nacional, pero, estamos trabajando con demasiadas comunas, es imposible comparar entre comunas con menos casos activos a lo largo del timepo, incluso es dificil comparar las comunas con mas casos activos, ya que, los colores que corresponden a cada comuna se repiten. Por otro lado, si separamos los graficos por region, podemos ver con mas claridad la evolución de las comunas, excepto en las regiones con muchas comunas, en este caso ocurre lo mismo que si graficamos todas las comunas en un grafico, pero, si quisieramos comparar comunas de distintas regiones tendriamos que trabajar con mas de un grafico y la escala de cada grafico puede causar algun error. 

#### Ejercicio 8

(10 puntos)

Hacer un gráfico que permita comparar rápidamente entre regiones su promedio de casos activos , máximo de casos confirmados y fallecidos. Utiliza los valores reales y apoyarlos con colores.

Se adjunta el diccionario `region_names` con tal de reemplazar los nombres de las regiones en los datos `fallecidos` para poder unir con los otros datos. 

In [ ]:
for pos in range(len(activos["region"].unique())):
    act = list(activos["region"].unique())
    fall = list(fallecidos["region"].unique())
    conf = list(confirmados["region"].unique())
    print(f"{act[pos]} -- {fall[pos]} -- {conf[pos]}")

In [ ]:
region_names = {
    "Tarapaca": "Tarapacá",
    "Valparaiso": "Valparaíso",
    "Metropolitana": "Metropolitana de Santiago",
    "O’Higgins": "Libertador General Bernardo O’Higgins", 
    "Libertador General Bernardo O'Higgins": "Libertador General Bernardo O’Higgins", 
    "Del Libertador General Bernardo O’Higgins": "Libertador General Bernardo O’Higgins",
    "Nuble": "Ñuble",
    "Biobio": "Biobío",
    "Araucanía": "La Araucanía",
    "La Araucania": "La Araucanía",
    "Los Rios": "Los Ríos",
    "Aysén": "Aysén del General Carlos Ibáñez del Campo",
    "Aysen": "Aysén del General Carlos Ibáñez del Campo",
    "Magallanes": "Magallanes y de la Antártica Chilena",
    "Magallanes y la Antartica": "Magallanes y de la Antártica Chilena"    
}

In [ ]:
activos_2=(
    activos.drop(columns=["codigo_region", "comuna", "codigo_comuna", "poblacion"])
    .replace({"region":region_names}, inplace=False)
)

confirmados_2=(
    confirmados.drop(columns=["region_id", "provincia_id", "provincia", "comuna_id", "comuna", "tasa", "poblacion"])
    .replace({"region":region_names}, inplace=False)
)

fallecidos_2=(
    fallecidos.replace({"region":region_names}, inplace=False)
)

df=(
    activos_2.merge(confirmados_2, on=["region", "fecha"])
    .merge(fallecidos_2, on=["region", "fecha"])
    .groupby(["region","fecha"])
    .sum().reset_index()
)

df 

**Comentarios:** 

## Ejercicio 9


En este ejercicio buscaremos realizar un mini-dashboard respecto al estado de los casos de COVID-19 en Chile, por lo tanto utilizaremos haremos uso de datos geográficos de manera operacional (es decir, no nos preocuparemos de proyecciones en mapas ni nada por el estilo), lo único es que debes instalar `geopandas` en tu ambiente virtual y no olvidar actualizarlo en tu `environment.yml` para luego subirlo a tu repositorio de GitHub.

Con tu ambiente activo (`conda activate mat281`) basta con ejecutar `conda install -c conda-forge geopandas` para instalar `geopandas`.

In [ ]:
import geopandas as gpd
from pathlib import Path

In [ ]:
shp_filepath = Path().resolve().parent / "data" / "regiones_chile.shp"
regiones = gpd.read_file(shp_filepath)
regiones

In [ ]:
type(regiones)

Lo único que tienes que saber es que un `GeoDataFrame` es idéntico a un `DataFrame` salvo que debe poseer una columna llamada `geometry` caracterice los elementros geométricos, que en este casos son polígonos con los límites de las regiones de Chile. 

Para graficar mapas en Altair se debe usar `mark_geoshape`, además, para no preocuparnos de las proyecciones si o si debes declarar lo siguiente que se muestra en la siguiente celda en las propiedades del gráfico. El resto es igual a cualquier otro gráfico de Altair.

In [ ]:
alt.Chart(regiones).mark_geoshape().encode(
).properties(
    projection={'type': 'identity', 'reflectY': True},
    width=250,
    height=600
)

### Ejercicio 9.1

(10 puntos)

Define el `DataFrame` con el nombre `casos_geo` tal que tenga las columnas

* `region`
* `codigo_region`
* `fecha`
* `poblacion`
* `casos_confirmados`
* `tasa`
* `casos_activos`
* `fallecidos`
* `geometry`

Ten mucho cuidado como unes los dataframes `confirmados`, `activos`, `fallecidos` y `regiones`. Idealmente utilizar el código de región, pero en caso que no se encuentren disponibles utilizar el nombre de la región (no olivdar utilizar el diccionario `region_names`).

In [ ]:
activos_3 = ( #Nuevo Dataframe de activos para hacer merge
    activos.drop(columns=["comuna", "codigo_comuna"])
    .groupby(["region", "codigo_region", "fecha"])
    .sum().reset_index()    
    .replace({"region":region_names}, inplace=False)
)

confirmados_3=( #Nuevo Dataframe de confirmados para hacer merge
    confirmados.drop(columns=["provincia_id", "provincia", "comuna", "comuna_id"])
    .rename(columns={"region_id": "codigo_region"})
    .groupby(["region", "codigo_region", "fecha"])
    .sum().reset_index()
    .replace({"region":region_names}, inplace=False)
)

fallecidos_3=(
    fallecidos.replace({"region":region_names}, inplace=False)
)

df_aux1=( #dataframe auxiliar para hacer merge con regiones 
    activos_3.merge(confirmados_3, on=["region", "fecha"])
   .drop(columns=["codigo_region_y", "poblacion_y"])
   .rename(columns={"codigo_region_x": "codigo_region", "poblacion_x": "poblacion"})
   .merge(fallecidos_3, on=["region", "fecha"])
)
    
regiones2=( #dataframe regiones sin columnas innecesarias para hacer merge con el datafraem auxiliar 
    regiones.rename(columns={"codregion": "codigo_region"})
    .drop(columns=["Region", "objectid", "area_km", "st_area_sh", "st_length_", "cir_sena"])
)

casos_geo = (
    df_aux1.merge(regiones2, on="codigo_region")
)

casos_geo.head()

Ejecuta lo siguiente para convertir el DataFrame anterior en un GeoDataFrames

In [ ]:
casos_geo = casos_geo.pipe(lambda x: gpd.GeoDataFrame(x, geometry="geometry"))

### Ejercicio 9.2

(5 puntos)

Modifica la función `covid_chile_chart` tal que reciba una fecha y una columna. Luego, debe filtrar `casos_geo` con registros de la fecha seleccionada y graficar un mapa donde las regiones se colereen según la columna escogida. 

In [ ]:
def covid_chile_chart(fecha, col):
    
    fecha = pd.to_datetime(fecha)
    data = casos_geo.loc[lambda x: x["fecha"] == fecha]
    
    chart = alt.Chart(data).mark_geoshape().encode(
        color=alt.Color(col, title=col.capitalize().replace("_", " "), scale=alt.Scale(scheme="lightgreyteal"))
    ).properties(
        projection={'type': 'identity', 'reflectY': True},
        width=150,
        height=400
    )
    
    chart.display()
    return

Prueba con lo siguiente

In [ ]:
fecha = "2020-04-13"
col = "tasa"
covid_chile_chart(fecha, col)

### Ejercicio 9.3

(5 puntos)

Ahora utilizando `widgets` generaremos el dashboard interactivo. Define lo siguiente:

* col_widget: Un `widgets.Dropdown` donde las opciones a seleccionar sean las columnas `poblacion`, `casos_confirmados`, `tasa`, `casos_activos` y `fallecidos`. Además, el argumento `description` debe ser `Columna`.
* fecha_widget: Un `widgets.DatePicker` donde el argumento `description` sea `Fecha`.
* Ambos widgets deben tener el argumento `continuous_update=False`

In [ ]:
import ipywidgets as widgets
from ipywidgets import interactive, interact

In [ ]:
col_widget = widgets.Dropdown(options=[("Población","poblacion"),("Casos Confirmados","casos_confirmados"),("Tasa","tasa"),("Casos Activos","casos_activos"),("Fallecidos","fallecidos")],
                             continous_update=False)

In [ ]:
fecha_inicial=pd.to_datetime("2020-04-13")

fecha_widget = widgets.DatePicker(
    description='Fecha',
    continous_update=False,
    value=fecha_inicial
)

Finalmente, haciendo uso de `interactive`, la función `covid_chile_chart` y todos los widgets es posible crear un _dashboard_ interactivo con los datos de Covid-19.

Respira profundo y explora tu creación!

In [ ]:
covid_dashboard = interactive(
    covid_chile_chart,
    fecha=fecha_widget,
    col=col_widget
)
covid_dashboard

**Comentarios:** El dashboard interactivo parece muy util si se quiere comparar distintas situaciones en una cierta fecha a nivel nacional.